In [1]:
import numpy as np
import cv2
from tqdm import tqdm
from tkinter import Tk, filedialog

In [2]:
import numpy as np
import cv2
from tqdm import tqdm
from tkinter import Tk, filedialog

ESCAPE_KEY_ASCII = 27

def onChange(value):
    pass

def process_image(img_bgr, contrast_value, brightness_value, use_clahe):
    img_float = img_bgr.astype(np.float32)
    contraste = contrast_value / 100.0
    ajustada = contraste * (img_float - 128) + 128 + brightness_value
    ajustada = np.clip(ajustada, 0, 255).astype(np.uint8)

    if not use_clahe:
        return ajustada

    lab = cv2.cvtColor(ajustada, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    l_clahe = clahe.apply(l)
    lab_clahe = cv2.merge((l_clahe, a, b))
    return cv2.cvtColor(lab_clahe, cv2.COLOR_LAB2BGR)

def resize_keep_aspect(img, scale):
    """Redimensiona mantendo a proporção"""
    h, w = img.shape[:2]
    new_w = int(w * scale)
    new_h = int(h * scale)
    return cv2.resize(img, (new_w, new_h), interpolation=cv2.INTER_LINEAR)

def show_centered(windowTitle, img, copyimg, apply_clahe, zoom_scale):
    """
    Mostra a imagem original e a processada, lado a lado,
    centralizadas em uma janela com fundo cinza.
    """
    # aplica zoom
    img_zoom = resize_keep_aspect(img, zoom_scale)
    copyimg_zoom = resize_keep_aspect(copyimg, zoom_scale)

    combined = np.hstack((img_zoom, copyimg_zoom))

    # Tamanho da tela
    screen_res = 1920, 1080
    try:
        import tkinter as tk
        root = tk.Tk()
        screen_res = root.winfo_screenwidth(), root.winfo_screenheight()
        root.destroy()
    except:
        pass

    win_w, win_h = screen_res

    # Fundo cinza (com altura extra p/ trackbars)
    canvas_h = max(combined.shape[0], win_h - 100)
    canvas_w = max(combined.shape[1], win_w)
    canvas = np.full((canvas_h, canvas_w, 3), 200, dtype=np.uint8)  # fundo cinza

    # Calcula posição central
    y_off = (canvas_h - combined.shape[0]) // 2
    x_off = (canvas_w - combined.shape[1]) // 2

    # Cola a imagem no centro
    canvas[y_off:y_off+combined.shape[0], x_off:x_off+combined.shape[1]] = combined

    # === textos ===
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 1.2
    thickness = 2
    color = (255, 255, 255)
    shadow_color = (50, 50, 50)
    y_pos = y_off + 40  # ajustado pela margem

    cv2.putText(canvas, "ANTES", (x_off + 30, y_pos), font, font_scale, shadow_color, thickness + 2, cv2.LINE_AA)
    cv2.putText(canvas, "ANTES", (x_off + 30, y_pos), font, font_scale, color, thickness, cv2.LINE_AA)
    cv2.putText(canvas, "DEPOIS", (x_off + img_zoom.shape[1] + 30, y_pos), font, font_scale, shadow_color, thickness + 2, cv2.LINE_AA)
    cv2.putText(canvas, "DEPOIS", (x_off + img_zoom.shape[1] + 30, y_pos), font, font_scale, color, thickness, cv2.LINE_AA)

    # Status CLAHE + Zoom
    clahe_status = f"CLAHE: {'ON' if apply_clahe else 'OFF'} | Zoom: {int(zoom_scale*100)}%"
    text_size, _ = cv2.getTextSize(clahe_status, font, font_scale, thickness)
    text_x = canvas.shape[1] - text_size[0] - 20
    text_y = canvas.shape[0] - 20
    cv2.putText(canvas, clahe_status, (text_x, text_y), font, font_scale, shadow_color, thickness + 2, cv2.LINE_AA)
    cv2.putText(canvas, clahe_status, (text_x, text_y), font, font_scale, color, thickness, cv2.LINE_AA)

    cv2.imshow(windowTitle, canvas)

# === Selecionar a Imagem ===
Tk().withdraw()
file_path = filedialog.askopenfilename(
    title="Selecione uma imagem",
    filetypes=[("Imagens", "*.jpg;*.jpeg;*.png;*.bmp;*.tiff")]
)

if not file_path:
    print("Nenhuma imagem selecionada. Encerrando...")
    exit()

img = cv2.imread(file_path)
if img is None:
    print("Erro ao carregar a imagem. Encerrando...")
    exit()

copyimg = img.copy()

# === Criar janela principal ===
windowTitle = "Ajuste de Brilho e Contraste (Antes e Depois)"
cv2.namedWindow(windowTitle, cv2.WINDOW_NORMAL)

# Trackbars
cv2.createTrackbar("Contraste", windowTitle, 100, 200, onChange)
cv2.createTrackbar("Brilho", windowTitle, 0, 200, onChange)

prev_contrast = cv2.getTrackbarPos("Contraste", windowTitle)
prev_brightness = cv2.getTrackbarPos("Brilho", windowTitle)
apply_clahe = True
force_update = True

# zoom inicial
zoom_scale = 1.2

while True:
    current_contrast = cv2.getTrackbarPos("Contraste", windowTitle)
    current_brightness = cv2.getTrackbarPos("Brilho", windowTitle)

    changed = (current_contrast != prev_contrast) or (current_brightness != prev_brightness)

    if changed or force_update:
        for _ in tqdm(range(1), desc="Aplicando ajustes", unit="passo"):
            copyimg = process_image(img, current_contrast, current_brightness, apply_clahe)

        prev_contrast = current_contrast
        prev_brightness = current_brightness
        force_update = False

    show_centered(windowTitle, img, copyimg, apply_clahe, zoom_scale)

    key = cv2.waitKey(1) & 0xFF
    if key == ESCAPE_KEY_ASCII:
        break
    elif key == ord('c'):
        apply_clahe = not apply_clahe
        force_update = True
        print(f"CLAHE {'ativado' if apply_clahe else 'desativado'}.")
    elif key == ord('r'):
        cv2.setTrackbarPos("Contraste", windowTitle, 100)
        cv2.setTrackbarPos("Brilho", windowTitle, 0)
        prev_contrast = 100
        prev_brightness = 0
        force_update = True
        print("🔄 Brilho e contraste resetados para os valores padrão.")
    elif key == ord('+') or key == ord('='):  # zoom in
        zoom_scale = min(3.0, zoom_scale + 0.1)  # até 300%
        force_update = True
        print(f"🔍 Zoom: {int(zoom_scale*100)}%")
    elif key == ord('-') or key == ord('_'):  # zoom out
        zoom_scale = max(0.5, zoom_scale - 0.1)  # mínimo 50%
        force_update = True
        print(f"🔍 Zoom: {int(zoom_scale*100)}%")

cv2.destroyAllWindows()

# === Salvar ===
Tk().withdraw()
save_path = filedialog.asksaveasfilename(
    defaultextension=".jpg",
    filetypes=[("JPEG", "*.jpg"), ("PNG", "*.png"), ("BMP", "*.bmp"), ("TIFF", "*.tiff")],
    title="Salvar imagem editada como..."
)

if save_path:
    cv2.imwrite(save_path, copyimg)
    print(f"✅ Imagem salva em: {save_path}")
else:
    print("❌ Imagem não salva.")

Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00,  4.76passo/s]


🔍 Zoom: 130%


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 58.77passo/s]


🔍 Zoom: 140%


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 76.91passo/s]


...

CLAHE ativado.


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 100.09passo/s]


🔄 Brilho e contraste resetados para os valores padrão.


Aplicando ajustes: 100%|██████████| 1/1 [00:00<00:00, 83.05passo/s]


✅ Imagem salva em: C:/Users/Victor Macedo/Desktop/sdsd.jpg
